<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb monte_carlo_v2.do.txt --encoding=utf-8 --ipynb_admon=hrule --ipynb_disable_mpl_inline --ipynb_cite=latex-plain -->

<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb monte_carlo.do.txt --encoding=utf-8 --ipynb_admon=hrule --ipynb_disable_mpl_inline --ipynb_cite=latex-plain

<!-- dom:TITLE: A brief introduction to UQ and SA with the Monte Carlo method -->
# A brief introduction to UQ and SA with the Monte Carlo method

**Vinzenz Gregor Eck**, Expert Analytics  
**Leif Rune Hellevik**, NTNU

First version: **Jul 13, 2018**

Updated: **Feb 02, 2025**

# Setup

In [1]:
# --- cell: install_chaospy ---
# @title Install chaospy (Colab-friendly)

try:
    import chaospy as cp
    import numpoly
    import numpy as np
    print("chaospy er allerede installert.")
except ImportError:
    # Installer chaospy fra PyPI. Dette drar inn numpoly automatisk.
    %pip install chaospy==4.3.21 --no-cache-dir
    import chaospy as cp
    import numpoly
    import numpy as np

print("numpy  :", np.__version__)
print("numpoly:", numpoly.__version__)
print("chaospy:", cp.__version__)

chaospy er allerede installert.
numpy  : 2.2.6
numpoly: 1.3.6
chaospy: 4.3.20


In [2]:
# --- cell: repo_setup ---
# @title Repo sync and environment setup

import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules
REMOTE = "https://github.com/lrhgit/uqsa2025.git"
REPO_PATH_COLAB = Path("/content/uqsa2025")

if IN_COLAB:
    if not REPO_PATH_COLAB.exists():
        print("Cloning repository...")
        subprocess.run(
            ["git", "clone", REMOTE, str(REPO_PATH_COLAB)],
            check=True
        )
    else:
        print("Updating existing repository...")
        subprocess.run(
            ["git", "-C", str(REPO_PATH_COLAB), "pull"],
            check=True
        )
    os.chdir(REPO_PATH_COLAB)

# --- Find repo root (works locally + in Colab) ---
cwd = Path.cwd().resolve()
repo_root = next(
    (p for p in [cwd] + list(cwd.parents) if (p / ".git").exists()),
    cwd
)

PY_SRC = repo_root / "python_source"
if PY_SRC.exists() and str(PY_SRC) not in sys.path:
    sys.path.insert(0, str(PY_SRC))

print("CWD:", Path.cwd())
print("repo_root:", repo_root)
print("python_source exists:", PY_SRC.exists())
print("python_source in sys.path:", str(PY_SRC) in sys.path)

CWD: /Users/leifh/git/uqsa2025
repo_root: /Users/leifh/git/uqsa2025
python_source exists: True
python_source in sys.path: True


In [3]:
# --- cell: layout_and_numpy_patch ---
# @title Layout fix, imports, and NumPy compatibility patch

import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML

HTML("""
<style>
div.cell.code_cell, div.output {
    max-width: 100% !important;
}
</style>
""")

import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp
import numpoly
import pandas as pd


# Pretty-print helpers (used across notebooks)
from pretty_printing import section_title, pretty_table, pretty_print_sobol_mc


# --- NumPy reshape compatibility patch for numpoly ---
_old_reshape = np.reshape

def _reshape_compat(a, *args, **kwargs):
    newshape = None
    if "newshape" in kwargs:
        newshape = kwargs.pop("newshape")
    if "shape" in kwargs and newshape is None:
        newshape = kwargs.pop("shape")
    if newshape is not None:
        return _old_reshape(a, newshape, *args, **kwargs)
    return _old_reshape(a, *args, **kwargs)

np.reshape = _reshape_compat
print("✓ numpy.reshape patched for numpoly compatibility")

✓ numpy.reshape patched for numpoly compatibility


In [4]:
# --- cell: load_problem_helpers ---
import numpy as np
import chaospy as cp

from sensitivity_examples_nonlinear import (
    analytic_sensitivity_coefficients,
)

# Monte Carlo

The Monte Carlo method (MCM)  is probably the most widely applied method for
variance based uncertainty quantification and sensitivity
analysis. Monte carlo methods are generally straight forward to use
and may be applied to a wide variety of problems as they require few
assumptions about the model or quantity of interest and require no
modifications of the model itself, i.e. the model may be used as a
black box. The basic idea is to calculate statistics (mean, standard
deviation, variance, sobol indices) of $Y$ directly from large amount
of sample evaluations from the black box model $y$.




**Monte Carlo approach.**

1. Sample a set of input samples $\mathbf{z}^{(s)}$ from the input space $\Omega_\mathbf{Z}$ that is defined by the joint probability density function ${F_Z}$.

2. Evaluate the deterministic model $y(\mathbf{z})$ for each sample in $\mathbf{z}^{(s)}$ to produce a set of model outputs $y^{(s)}$.

3. Estimate all uncertainty measures and sensitivity indices from $y^{(s)}$.
<hr/>



For demonstration purposes we will use the same model as before:

In [5]:
# --- cell: linear_model ---

import numpy as np

def linear_model(w, z):
    """
    Linear model used across the entire notebook:
        Y = sum_i (w_i * z_i)

    Parameters
    ----------
    w : array-like, shape (Nrv,)
        Weights for each random variable.

    z : array-like, shape (Ns, Nrv)
        Sample matrix where rows are samples and columns correspond to random variables.

    Returns
    -------
    Y : array-like, shape (Ns,)
        Model outputs for each sample.
    """
    return np.sum(w * z, axis=1)

### Expectation and variance

Once the model outputs have been computed the expectation and variance
of the output are computed with the normal estimators.


$$
    {\mathbb{E}}(Y) \approx \frac{1}{N} \sum_{s=1}^{N} y^{(s)} \qquad \text{and} \qquad       \operatorname{Var}(Y) \approx \frac{1}{N\!-\!1} \sum_{s=1}^{N}  \left( y^{(s)} - {\mathbb{E}}(Y)\right)^2.
$$

In [6]:
# --- cell: define_problem_and_jpdf ---
# Define the (Z, W) non-additive linear demo problem and construct the joint distribution explicitly

Nrv = 4  # number of Z-variables (and also number of W-variables)

# Each row is [mu, sigma] for a Normal(mu, sigma)
zm = np.array([[0.0, i] for i in range(1, Nrv + 1)])

c = 0.5
wm = np.array([[i * c, i] for i in range(1, Nrv + 1)])

# --- Explicit construction of the joint PDF (independent marginals) ---
# Stack Z and W parameter definitions -> total parameter vector X = (Z1..ZNrv, W1..WNrv)
x_params = np.vstack([zm, wm])  # shape (2*Nrv, 2)

marginals = [cp.Normal(mu, sigma) for (mu, sigma) in x_params]
jpdf = cp.J(*marginals)  # independent joint distribution


print("Nrv =", Nrv, " -> total parameters =", len(jpdf))

Nrv = 4  -> total parameters = 8


In [7]:
# 1. Generate a set of Xs
Ns = 20000
Xs = jpdf.sample(Ns, rule='R').T  # <- transform the sample matrix

# 2. Evaluate the model
Zs = Xs[:, :Nrv]
Ws = Xs[:, Nrv:]
Ys = linear_model(Ws, Zs)

# 3. Calculate expectation and variance
EY = np.mean(Ys)
VY = np.var(Ys, ddof=1)  # NB: use ddof=1 for unbiased variance estimator, i.e /(Ns - 1)

print('E(Y): {:2.5f} and  Var(Y): {:2.5f}'.format(EY, VY))

E(Y): -0.09588 and  Var(Y): 443.55090


### Variance based sensitivity measures



### Monte Carlo estimation of expectation and variance

We now illustrate the basic Monte Carlo procedure for uncertainty quantification. Random samples are drawn from the joint input distribution, the model is evaluated for each sample, and simple sample statistics are used to approximate the expectation and variance of the model output.

In the next cell, we also compute the **analytical Sobol sensitivity indices**, which will be used as reference values for assessing the accuracy of the Monte Carlo estimates introduced below.

In [8]:
# Analytic sensitivity values (reference)
Sa, Szw, Sta = analytic_sensitivity_coefficients(zm, wm)


In our [sensitivity_introduction notebook](sensitivity_introduction.ipynb) model we calculated the sensitivity
coefficients with the MCM in the following manner:


### Making the Monte Carlo algorithm explicit

Up to this point, the Monte Carlo sensitivity analysis has been carried out using a single high-level function call,

```python
A_s, B_s, C_s, f_A, f_B, f_C, Smc, Stmc = monte_carlo_sens_nonlin(Ns_mc, jpdf)
```

which hides how the Sobol indices are actually computed.
While this abstraction is convenient, it is important to understand **what happens under the hood** and why the algorithm is structured the way it is.

In the following, we therefore unpack the method and explicitly construct the sampling matrices, following **Saltelli’s algorithm** for variance-based sensitivity analysis.

---

### Why a special algorithm is needed

A direct (“brute-force”) Monte Carlo estimation of Sobol indices is computationally impractical.
Consider the first-order Sobol index
$$
S_i = \frac{\operatorname{Var}\big(\mathbb{E}[Y \mid Z_i]\big)}{\operatorname{Var}(Y)} .
$$

Estimating the conditional expectation
$\mathbb{E}[Y \mid Z_i] $
for a fixed value of $Z_i$ requires many model evaluations.
Repeating this over the full range of $Z_i$, and for all input variables, leads to a total cost that scales like
$$
\mathcal{O}(M^2, r),
$$
which quickly becomes infeasible. Even moderate sample sizes can imply millions of model evaluations.

---

### Saltelli’s key idea

Saltelli’s insight is that **all first-order and total Sobol indices can be estimated simultaneously** using a carefully structured reuse of model evaluations.
This reduces the total cost to
$$
M,(r + 2)
$$
model evaluations, making global sensitivity analysis feasible in practice.

---

### Saltelli’s algorithm (conceptual steps)

1. Generate two independent Monte Carlo sample matrices $A$ and $B$.
2. Construct mixed matrices $C_i$, where all columns come from $A$ except the $i$-th, which is taken from $B$.
3. Evaluate the model for all samples in $A$, $B$, and $C_i$.
4. Combine these evaluations to estimate the Sobol sensitivity indices.

---

In the next cell, we explicitly construct the matrices $A$, $B$, and $C_i$ and verify their structure before using them to compute the Sobol indices.

### Explicit construction of Saltelli sample matrices

We now explicitly construct the Saltelli sampling matrices used in the Monte Carlo
estimation of Sobol indices.

The construction is as follows:

- Draw \(2N_s\) independent samples from the joint distribution
- Split these into two matrices \(A\) and \(B\)
- Construct mixed matrices \(C_i\), where all columns come from \(A\)
  except column \(i\), which is taken from \(B\)

This explicit construction allows us to **verify the structure** of the matrices
before using them to compute Sobol sensitivity indices.


In [9]:
# ------------------------------------------------------------
# Explicit construction of Saltelli sample matrices
# ------------------------------------------------------------

Ns_mc = 1000
P = len(jpdf)  # total number of input parameters

# Step 1: draw 2*Ns samples
Xtot = jpdf.sample(2 * Ns_mc, rule="R").T  # shape: (2*Ns, P)

# Step 2: split into A and B
A_s = Xtot[:Ns_mc, :]
B_s = Xtot[Ns_mc:, :]

# Step 3: construct C_i matrices
C_s = np.empty((P, Ns_mc, P))

for i in range(P):
    C_s[i, :, :] = A_s.copy()
    C_s[i, :, i] = B_s[:, i].copy()

# ------------------------------------------------------------
# Inspect structure (do NOT print full matrices)
# ------------------------------------------------------------

print("Shapes:")
print("A_s:", A_s.shape)
print("B_s:", B_s.shape)
print("C_s:", C_s.shape)

print("\nFirst 5 rows of A_s:")
display(pd.DataFrame(A_s[:5, :], columns=[f"X{j+1}" for j in range(P)]))

print("\nFirst 5 rows of B_s:")
display(pd.DataFrame(B_s[:5, :], columns=[f"X{j+1}" for j in range(P)]))

# Structural check for one C_i
i = 0
print(f"\nStructural check for C_{i}:")
print(
    "Columns equal to A except i?",
    np.allclose(
        np.delete(C_s[i], i, axis=1),
        np.delete(A_s, i, axis=1)
    )
)
print(
    "Column i equals B?",
    np.allclose(C_s[i][:, i], B_s[:, i])
)


Shapes:
A_s: (1000, 8)
B_s: (1000, 8)
C_s: (8, 1000, 8)

First 5 rows of A_s:


,X1,X2,X3,X4,X5,X6,X7,X8
0,0.088772,0.604149,-6.611834,-1.651293,0.665165,2.349298,-0.178815,9.141922
1,0.134207,-3.887563,3.180010,-5.893275,-0.352175,-1.044655,5.530543,2.227072
2,-0.043429,-0.531401,-3.521622,-2.987505,0.310575,3.429128,-0.099439,2.110188
3,-0.675899,-0.271552,-3.834529,-11.302929,1.537966,-0.244643,1.524923,5.938042
4,-2.222773,1.955196,1.009247,0.490925,0.743668,-0.559305,3.742341,5.653095



First 5 rows of B_s:


,X1,X2,X3,X4,X5,X6,X7,X8
0,-0.439022,-0.247374,-3.578170,-6.063943,1.942883,-1.676798,-1.150729,-4.146891
1,-1.533902,1.003144,-2.849454,8.795852,1.665859,0.204052,-0.915038,2.051723
2,1.855066,-0.262323,-0.484242,5.475592,1.675383,2.251763,-2.560556,14.411188
3,-1.205245,-0.313174,0.318247,-5.585745,-0.373181,1.863756,3.259556,-1.354099
4,-1.165563,-1.178785,-0.825572,3.007825,-0.445937,2.394794,-2.956613,4.861381



Structural check for C_0:
Columns equal to A except i? True
Column i equals B? True


### Step 2: explicit model evaluation on Saltelli matrices

Having constructed the Saltelli sample matrices \(A\), \(B\), and \(C_i\),
we now explicitly evaluate the model for each of them.

We deliberately avoid introducing an additional wrapper function such as
`eval_model`. Since the model is already defined as `linear_model`,
an explicit evaluation keeps the algorithm transparent and makes the
connection to the mathematical formulation clearer:

$$
Y_A = f(A), \qquad
Y_B = f(B), \qquad
Y_{C_i} = f(C_i).
$$

This step corresponds to **Step 2 in Saltelli’s algorithm**.


In [10]:
# ------------------------------------------------------------
# Step 2: evaluate the model on Saltelli matrices
# ------------------------------------------------------------

N_terms = int(P / 2)  # number of Z-variables (assumed stacked first)

# Evaluate model for A and B
Y_A = linear_model(
    A_s[:, N_terms:],   # w
    A_s[:, :N_terms]    # z
)

Y_B = linear_model(
    B_s[:, N_terms:],   # w
    B_s[:, :N_terms]    # z
)

# Evaluate model for each C_i
Y_C = np.zeros((P, Ns_mc))

for i in range(P):
    Xi = C_s[i]
    Y_C[i, :] = linear_model(
        Xi[:, N_terms:],   # w
        Xi[:, :N_terms]    # z
    )

# Sanity check
print("Shapes after model evaluation:")
print("Y_A:", Y_A.shape)
print("Y_B:", Y_B.shape)
print("Y_C:", Y_C.shape)


Shapes after model evaluation:
Y_A: (1000,)
Y_B: (1000,)
Y_C: (8, 1000)


### Step 3: estimation of Sobol sensitivity indices

Having evaluated the model on the Saltelli sample matrices,
we now estimate the **first-order** and **total** Sobol sensitivity indices.

In the expressions below we adopt the notation

$$Y_A = f(A), \qquad Y_B = f(B), \qquad Y_{C_i} = f(C_i).$$

Some texts instead write $f_A, f_B, f_{C_i}$; this is purely a matter of notation — the quantities are identical.

Using (Y_A, Y_B, Y_{C_i}) makes the connection to the model evaluations in the previous cell explicit and consistent with the variables we actually computed.


Using Saltelli’s Monte Carlo estimators, the indices are computed as

$$
S_i \;=\;
\frac{\frac{1}{N_s}\sum_{s=1}^{N_s}
Y_B^{(s)}\,
\bigl(Y_{C_i}^{(s)} - Y_A^{(s)}\bigr)}
{\operatorname{Var}(Y)},
$$

and

$$
S_{T_i} \;=\;
\frac{\frac{1}{2N_s}\sum_{s=1}^{N_s}
\bigl(Y_A^{(s)} - Y_{C_i}^{(s)}\bigr)^2}
{\operatorname{Var}(Y)}.
$$

These estimators reuse the same model evaluations from
\(A\), \(B\), and \(C_i\), making the computation efficient.

This step corresponds to **Step 3 in Saltelli’s algorithm**.


For parameters with very small first-order effects, the Monte Carlo estimators
may exhibit noticeable sampling variability even when the underlying theory is
exact. This is particularly true for interaction-dominated parameters.
While the total-order index \(S_{T_i}\) is generally more robust than the
first-order index \(S_i\), it may also show finite-sample variability for
moderate sample sizes.


In [11]:
# ------------------------------------------------------------
# Step 3: compute Sobol indices using Saltelli estimators
# ------------------------------------------------------------

# Estimate variance from A-samples
Var_Y = np.var(Y_A, ddof=1)

S_mc  = np.zeros(P)
ST_mc = np.zeros(P)

for i in range(P):
    # First-order Sobol index
    S_mc[i] = np.mean(
        Y_B * (Y_C[i] - Y_A)
    ) / Var_Y

    # Total Sobol index
    ST_mc[i] = np.mean(
        (Y_A - Y_C[i])**2
    ) / (2 * Var_Y)

# Display results
row_labels = [f"X{i+1}" for i in range(P)]

df_sobol = pd.DataFrame(
    {
        "S (MC)": S_mc,
        "ST (MC)": ST_mc,
    },
    index=row_labels
)

display(section_title("Monte Carlo Sobol indices (explicit Saltelli)"))
pretty_table(df_sobol, floatfmt=".3f")


,S (MC),ST (MC)
X1,0.000,0.003
X2,0.013,0.047
X3,0.047,0.260
X4,0.158,0.765
X5,-0.002,0.002
X6,0.002,0.035
X7,0.054,0.195
X8,-0.003,0.590


To verify that the explicit construction and estimation carried out above
are correct, it is useful to compare the resulting Sobol indices with those
obtained earlier using the higher-level implementation.

In the next cell, we therefore collect the analytical, Monte Carlo, and
polynomial chaos estimates of the sensitivity indices in a single table.
This provides a compact consistency check and makes it easier to assess
agreement across methods.


In [12]:

from monte_carlo import calculate_sensitivity_indices_mc
# Alias (do NOT modify variables)
S_mc_explicit  = S_mc.copy()
ST_mc_explicit = ST_mc.copy()

# Keep notation consistent with the earlier black-box implementation:
f_A, f_B, f_C = Y_A, Y_B, Y_C



Smc, Stmc = calculate_sensitivity_indices_mc(f_A, f_B, f_C,clip=False)
S_mc_blackbox   = Smc.copy()
ST_mc_blackbox  = Stmc.copy()



labels = (
    [f"Z_{i}" for i in range(1, Nrv+1)] +
    [f"W_{i}" for i in range(1, Nrv+1)]
)

# Build comparison table
df_compare = pd.DataFrame(
    {
        "S (MC explicit)":   S_mc_explicit,
        "S (MC blackbox)":   S_mc_blackbox,
        "ΔS":                S_mc_blackbox - S_mc_explicit,
        "ST (MC explicit)":  ST_mc_explicit,
        "ST (MC blackbox)":  ST_mc_blackbox,
        "ΔST":               ST_mc_blackbox - ST_mc_explicit,
    },
    index=labels
)

display(section_title("Monte Carlo Sobol indices: explicit vs blackbox"))
pretty_table(df_compare, floatfmt=".3f")


,S (MC explicit),S (MC blackbox),ΔS,ST (MC explicit),ST (MC blackbox),ΔST
Z_1,0.000,0.000,0.000,0.003,0.003,0.000
Z_2,0.013,0.013,0.000,0.047,0.047,0.000
Z_3,0.047,0.047,0.000,0.260,0.260,0.000
Z_4,0.158,0.158,0.000,0.765,0.765,0.000
W_1,-0.002,-0.002,0.000,0.002,0.002,0.000
W_2,0.002,0.002,0.000,0.035,0.035,0.000
W_3,0.054,0.054,0.000,0.195,0.195,0.000
W_4,-0.003,-0.003,0.000,0.590,0.590,0.000
